# 1. 导入数据

In [ ]:
import pandas as pd

# 导入数据集
movies_df = pd.read_csv('C:\VSCode\Visualization\Movie\Movie_Actors.csv', encoding='latin1')
actors_df = pd.read_csv('C:\VSCode\Visualization\Movie\Movie_Actors.csv', encoding='latin1')
ratings_df = pd.read_csv('C:\VSCode\Visualization\Movie\Movie_AdditionalRating.csv', encoding='latin1')
genres_df = pd.read_csv('C:\VSCode\Visualization\Movie\Movie_Genres.csv', encoding='latin1')
writers_df = pd.read_csv('C:\VSCode\Visualization\Movie\Movie_Writer.csv', encoding='latin1')

# 2.合并数据集

## 2.1 合并Movie_Movies.csv和Movie_Actors.csv

In [ ]:
# 合并电影和演员信息
merged_df = pd.merge(movies_df, actors_df, on='imdbID', how='left')

## 2.2 合并Movie_AdditionalRating.csv

In [ ]:
# 合并电影和评分信息
merged_df = pd.merge(merged_df, ratings_df, on='imdbID', how='left')

## 2.3 合并Movie_Genres.csv

In [ ]:
# 合并电影和流派信息
merged_df = pd.merge(merged_df, genres_df, on='imdbID', how='left')

## 2.4 合并Movie_Writer.csv

In [ ]:
# 合并电影和编剧信息
merged_df = pd.merge(merged_df, writers_df, on='imdbID', how='left', suffixes=('_movies', '_writers'))

# 3. 保存合并后的数据

In [ ]:
# 保存处理后的数据
merged_df.to_csv('movie_data.csv', index=False)

# 4. 合并数据

## 4.1 查看数据结构

In [ ]:
# 读取合并后的数据文件
merged_df = pd.read_csv('movie_data.csv')

# 查看数据的基本信息（包括列名、数据类型、非空值数量等）
print("数据概览：")
print(merged_df.info())

# 查看数据的前5行
print("\n数据的前5行：")
print(merged_df.head())

# 查看每列的缺失值数量
print("\n每列缺失值数量：")
print(merged_df.isnull().sum())

# 查看每列的唯一值数量
print("\n每列的唯一值数量：")
print(merged_df.nunique())

## 4.2 删除冗余列

In [ ]:
# 删除不需要的冗余列
merged_df = merged_df.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'Unnamed: 0_movies', 'Unnamed: 0_writers'])

## 4.3 处理缺失值

In [ ]:
# 删除对分析不重要的列，如 Awards, Plot, Poster, Production
merged_df = merged_df.drop(columns=['Awards', 'Plot', 'Poster', 'Production'])

# 填充缺失的“Rating”和“Director”列
merged_df['Rating'] = merged_df['Rating'].fillna('unknown')
merged_df['Director'] = merged_df['Director'].fillna('unknown')

# 对于"imdbRating"和"Released"列，可以填充为0或“未知”，也可以删除含有缺失值的行
merged_df['imdbRating'] = merged_df['imdbRating'].fillna(0)
merged_df['Released'] = merged_df['Released'].fillna('unknown')

## 4.4 转换数据类型

In [ ]:
# 将“Year”列转换为整数类型
merged_df['Year'] = pd.to_numeric(merged_df['Year'], errors='coerce')

# 提取“Runtime”中的数字并将其转换为整数类型
merged_df['Runtime'] = merged_df['Runtime'].str.extract('(\d+)').astype(float)

# 检查并删除无效的值（例如年份为空或时长小于0）
merged_df = merged_df.dropna(subset=['Year', 'Runtime'])
merged_df = merged_df[merged_df['Runtime'] >= 0]

## 4.5 处理类别数据

In [ ]:
# 处理多个流派：拆分为多个列
genres_split = merged_df['Genre'].str.split(',', expand=True)

# 合并拆分的列并将其命名
for i in range(genres_split.shape[1]):
    merged_df[f'Genre_{i+1}'] = genres_split[i]

# 删除原始的“Genre”列
merged_df = merged_df.drop(columns=['Genre'])

## 4.6 处理“imdbVotes”列

In [ ]:
# 处理“imdbVotes”列，去除非数字字符（如逗号），然后转换为整数类型
merged_df['imdbVotes'] = merged_df['imdbVotes'].str.replace(',', '').astype(float)

# 处理可能存在的缺失值（可以填充为0，或删除）
merged_df['imdbVotes'] = merged_df['imdbVotes'].fillna(0)

## 4.7 清洗无效数据

In [ ]:
# 处理“Rated”列，保留有效的评级值
valid_ratings = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Unrated']  # 示例，你可以根据实际需要更新
merged_df = merged_df[merged_df['Rated'].isin(valid_ratings)]

# 删除“Website”列中缺失或无效的数据
merged_df['Website'] = merged_df['Website'].fillna('未知')

## 4.8 输出处理后的数据

In [ ]:
# 保存清理后的数据
merged_df.to_csv('cleaned_movie_data.csv', index=False)

# 5. 验证数据集

## 5.1 检查数据完整性

In [ ]:
# 查看数据中每列的缺失值情况
print(merged_df.isnull().sum())

In [ ]:
# 填充缺失的“Country”和“Language”列
merged_df['Country'] = merged_df['Country'].fillna('unknown')
merged_df['Language'] = merged_df['Language'].fillna('unknown')

# 填充“Actors”列，填充为"未知"
merged_df['Actors'] = merged_df['Actors'].fillna('unknown')

# 填充“RatingSource”列，填充为"未知"
merged_df['RatingSource'] = merged_df['RatingSource'].fillna('unknown')

# 对于“Person”和“Responsibility”列，删除缺失值的行
merged_df = merged_df.dropna(subset=['Person', 'Responsibility'])

# 填充“Genre_1”列，填充为"未知"
merged_df['Genre_1'] = merged_df['Genre_1'].fillna('unknown')

In [ ]:
# 删除包含缺失“imdbRating”或“imdbVotes”列的行
merged_df = merged_df.dropna(subset=['imdbRating', 'imdbVotes'])

In [ ]:
# 删除“Person”和“Responsibility”列（如果不重要）
merged_df = merged_df.drop(columns=['Person', 'Responsibility'])

## 5.2 检查数据结构是否适合可视化

In [ ]:
# 检查是否有重复的 imdbID（每个电影应该只有一行）
print(merged_df['imdbID'].duplicated().sum())

## 5.3 数据格式检查

In [ ]:
# 将“Released”列转换为日期格式
merged_df['Released'] = pd.to_datetime(merged_df['Released'], errors='coerce')

## 5.4 列名规范化

In [ ]:
# 重命名列以符合规范
merged_df = merged_df.rename(columns={
    'imdbRating': 'imdb_rating',
    'imdbVotes': 'imdb_votes',
    'Released': 'release_date',
    'Runtime': 'runtime_minutes',
    'Country': 'country',
    'Language': 'language',
    'Title': 'movie_title',
    'Rating': 'rating'
})

## 5.5 数据类型一致性

In [ ]:
# 确保“imdbRating”和“imdbVotes”列为数值型
merged_df['imdb_rating'] = pd.to_numeric(merged_df['imdb_rating'], errors='coerce')
merged_df['imdb_votes'] = pd.to_numeric(merged_df['imdb_votes'], errors='coerce')

# 确保“Year”和“Released”是日期类型
merged_df['Year'] = pd.to_numeric(merged_df['Year'], errors='coerce')
merged_df['release_date'] = pd.to_datetime(merged_df['release_date'], errors='coerce')

## 5.8 导出数据并导入Tableau

In [ ]:
# 保存清理后的数据
merged_df.to_csv('cleaned_movie_data_for_tableau.csv', index=False)